In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection()
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

In [ ]:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

# Carga de Función en un estado cuántico.

En este caso partiendo de un estado $|x\rangle_{n}\otimes|0\rangle_{1}$  aplicamos la siguiente transformación:

$$|x\rangle_{n}\hat{R}_{y}(2\theta_{x})|0\rangle_{1}$$
$$|x\rangle_{n}\otimes(\cos{\theta_{x}}|0\rangle_{1} + \sin{\theta_{x}}|1\rangle_{1})$$

Donde $$\theta_{x}=\arccos{\sqrt{f(x)}}$$

Siendo $f(x)$ la función que queremos cargar en el estado cuántico. Esta función tiene que estar correctamente discretizada.

La idea es generar en n qbits una superposición de estados y aplicar las transformación anterior:

Es decir partimos de 

$$|\Psi\rangle_{n}\otimes |0\rangle_{1} = \frac{1}{2^{n/2}}\sum_{x=0}^{2^{n}-1}|x\rangle \otimes |0\rangle_{1}$$

y aplicamos la anterior transformación a todos y cada uno de los estados base anterior

$$\frac{1}{2^{n/2}}\sum_{x=0}^{2^{n}-1}|x\rangle\hat{R}_{y}(2\theta_{x})|0\rangle_{1}$$

Es decir tenemos que sumar todas las posibles rotaciones controladas

## 1.Implementación

In [ ]:
%load_ext qat.core.magic

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

In [ ]:
Path = '/home/cesga/gferro/NEASQC/PhaseAmplification/'
sys.path.append(Path)

### 1.1. Inicialización

Debemos discretizar el dominio de la función y obtener los valores de la función sobre dicha discretización. Como nos va a interesar que el dominio de la probabilidad y de la función sea el mismo utilizamos la función **get_histogram** para obtener una discretización del dominio que nos valga para la probabilidad y para la función que queremos cargar!!

In [ ]:
#Función de probabilidad de prueba
def p(x):
    return x*x

def f(x):
    return np.sin(x)
    

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins

In [ ]:
nqbits = 4
nbins = 2**nqbits
a = 0
b = 1

In [ ]:
#Creamos la probabilidad discretizada que queremos cargar
centers, probs = get_histogram(p, a, b, nbins)
#Queremos esto porque lo que me interesa es discretizar la función de forma
#acoplada a las probabilidades. Así pues para discretizar la función utilizamos
#el mismo dominio que el de la probabilidad.
DiscretizedFunction = p(centers)

In [ ]:
plt.plot(centers, probs, 'o')
plt.plot(centers, DiscretizedFunction, 'o')
plt.legend(['DiscretizedProbability', 'DiscretizedFunction'])

Se puede aprovechar mucho del trabajo realizado para la carga de la probabilidades. Ahora todo es un poco más sencillo (creo) ya que lo que tenemos que hacer es una suma de rotaciones controladas.

In [ ]:
#calculamos los ángulos que tenemos que rotar
Thetas = 2.0*np.arcsin(np.sqrt(DiscretizedFunction))

In [ ]:
from qat.core.console import display
from qat.lang.AQASM import Program, QRoutine, H, AbstractGate
from dataloading_module import CRBS_gate

In [ ]:
nqbits = 4
nbins = 2**nqbits
a = 0
b = 1

#Creamos la probabilidad discretizada que queremos cargar
centers, probs = get_histogram(p, a, b, nbins)
#Queremos esto porque lo que me interesa es discretizar la función de forma
#acoplada a las probabilidades. Así pues para discretizar la función utilizamos
#el mismo dominio que el de la probabilidad.
DiscretizedFunction = f(centers)

#calculamos los ángulos que tenemos que rotar
Thetas = 2.0*np.arcsin(np.sqrt(DiscretizedFunction))


qprog = Program()
qcontrol = qprog.qalloc(nqbits+1)
#qtarget = qprog.qalloc(1)
#Generamos la superposición de estados para un estado de nqbits
for i in range(nqbits):
    qprog.apply(H, qcontrol[i])

#iteramos sobre todos los posibles estados de nqbits    
for i in range(nbins):
    #estado |i>
    controlledR_gate = CRBS_gate(nqbits, i, Thetas[i])    
    qprog.apply(controlledR_gate, qcontrol)

In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit
#%qatdisplay circuit --depth 1

In [ ]:
#Create a Job from the circuit
job = circuit.to_job(qubits = [nqbits])
#job = circuit.to_job()

#Submit the job to the simulator LinAlg and get the results
result = lineal_qpu.submit(job)

In [ ]:
PResults = PostProcessResults(result.join())

In [ ]:
PResults

In [ ]:
print('Quantum Measurement: {}'.format(PResults['Probability'][1]*2**nqbits))
print('Integral: {}'.format(sum(DiscretizedFunction)))
np.isclose(PResults['Probability'][1]*2**nqbits, sum(DiscretizedFunction))

In [ ]:
np.isclose(PResults['Probability'][1]*2**nqbits, sum(DiscretizedFunction))

## 2.Creamos Puerta de Carga de la Función

In [ ]:
def R_generator(Dictionary):
    FunctionArray = Dictionary['array']
    nqbits_ = TestBins(FunctionArray, 'Function')
    #Calculation of the rotation angles
    Thetas = 2.0*np.arcsin(np.sqrt(FunctionArray))    
    
    qrout = QRoutine()
    qbits = qrout.new_wires(nqbits_+1)
    NumberOfStates = 2**nqbits_
    #Loop over the States
    for i in range(NumberOfStates):
        #State |i>    
        #Generation of a Controlled rotation of theta by state |i>
        controlledR_gate = CRBS_gate(nqbits_, i, Thetas[i])    
        qrout.apply(controlledR_gate, qbits)
    return qrout    

LoadR_Gate = AbstractGate(
    "R_Gate", 
    [dict],
    circuit_generator = R_generator,
    arity = lambda x:TestBins(x['array'], 'Function')+1
)        

In [ ]:
nqbits = 8
nbins = 2**nqbits
a = 0
b = 1

#Creamos la probabilidad discretizada que queremos cargar
centers, probs = get_histogram(p, a, b, nbins)
#Queremos esto porque lo que me interesa es discretizar la función de forma
#acoplada a las probabilidades. Así pues para discretizar la función utilizamos
#el mismo dominio que el de la probabilidad.
DiscretizedFunction = f(centers)

qprog = Program()
qcontrol = qprog.qalloc(nqbits+1)
#qtarget = qprog.qalloc(1)
#Generamos la superposición de estados para un estado de nqbits
for i in range(nqbits):
    qprog.apply(H, qcontrol[i])

R_gate = LoadR_Gate({'array': DiscretizedFunction})
qprog.apply(R_gate, qcontrol)

In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit
#%qatdisplay circuit --depth 2

In [ ]:
#Create a Job from the circuit
job = circuit.to_job(qubits = [nqbits])
result = lineal_qpu.submit(job)

In [ ]:
PR2_results = PostProcessResults(result)#.join())
print('Quantum Measurement: {}'.format(PR2_results['Probability'][1]*2**nqbits))
print('Integral: {}'.format(sum(DiscretizedFunction)))
np.isclose(PR2_results['Probability'][1]*2**nqbits, sum(DiscretizedFunction))

In [ ]:
print('Quantum Measurement: {}'.format(PR2_results['Probability'][1]*2**nqbits))
print('Integral: {}'.format(sum(DiscretizedFunction)))
np.isclose(PR2_results['Probability'][1]*2**nqbits, sum(DiscretizedFunction))

## 3.Probamos la función en fichero python

La funcion que crea la puerta de carga la incorporo al fichero **dataloading_module.py** con el fin de tener todas las utilidades de carga de datos bien unificadas.

In [ ]:
Path = '/home/cesga/gferro/NEASQC/PhaseAmplification/'
sys.path.append(Path)

In [ ]:
from AuxiliarFunctions import get_histogram
from dataloading_module import LoadR_Gate

In [ ]:
from qat.lang.AQASM import Program, H

In [ ]:
nqbits = 4
nbins = 2**nqbits
a = 0
b = 1

#Creamos la probabilidad discretizada que queremos cargar
centers, probs = get_histogram(p, a, b, nbins)
#Queremos esto porque lo que me interesa es discretizar la función de forma
#acoplada a las probabilidades. Así pues para discretizar la función utilizamos
#el mismo dominio que el de la probabilidad.
DiscretizedFunction = f(centers)
qprog = Program()
qcontrol = qprog.qalloc(nqbits+1)
#qtarget = qprog.qalloc(1)
#Generamos la superposición de estados para un estado de nqbits
for i in range(nqbits):
    qprog.apply(H, qcontrol[i])

R_gate = LoadR_Gate({'array': DiscretizedFunction})
qprog.apply(R_gate, qcontrol)

In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
%qatdisplay circuit
#%qatdisplay circuit --depth 2

In [ ]:
#Create a Job from the circuit
job = circuit.to_job(qubits = [nqbits])
result = lineal_qpu.submit(job)

In [ ]:
PR2_results = PostProcessResults(result.join())
print('Quantum Measurement: {}'.format(PR2_results['Probability'][1]*2**nqbits))
print('Integral: {}'.format(sum(DiscretizedFunction)))
np.isclose(PR2_results['Probability'][1]*2**nqbits, sum(DiscretizedFunction))

## 4. Programa Juan

Quiero comprobar que mi implementación es consistente con la implementación realizada en el programa de Juan

In [ ]:
sys.path.append('/home/gferro/Code/ProgramasDefinitivos')

In [ ]:
from expectation_module import load_function

In [ ]:
nqbits = 6
nbins = 2**nqbits
a = 0
b = 1

#Creamos la probabilidad discretizada que queremos cargar
centers, probs = get_histogram(p, a, b, nbins)
R_gate, y = load_function(centers, p, nqbits)

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits+1)
for i in range(nqbits):
    qprog.apply(H, qbits[i])
qprog.apply(R_gate, qbits)    

In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()

#Display the circuit
#%qatdisplay circuit
%qatdisplay circuit --depth 0

In [ ]:
#Create a Job from the circuit
job = circuit.to_job(qubits = [nqbits])

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

In [ ]:
QP = []
States = []
#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))
    QP.append(sample.probability)
    States.append(str(sample.state))
print('Quantum Measurement: {}'.format(QP[1]*2**(nqbits)))    
print('Integral: {}'.format(sum(y)))    

In [ ]:
np.isclose(QP[1]*2**(nqbits), sum(DiscretizedFunction))

In [ ]:
States = pd.Series(States)
QP = pd.Series(QP)

In [ ]:
np.isclose(sum(QP[States.str.extract(r'(\d)\>')[0] == '1'])*(2**(nqbits)),sum(DiscretizedFunction))

In [ ]:
sum(QP[States.str.extract(r'(\d)\>')[0] == '0'])*(2**(nqbits))

In [ ]:
sum(DiscretizedFunction)